In [21]:
import os 

from PIL import Image

import torch 
import torch.onnx
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [23]:

class PneumoniaDataset(Dataset):
    def __init__(self,root_dir,transform = None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        
        for label in ['NORMAL', 'PNEUMONIA']:
            class_dir = os.path.join(root_dir,label)
            for image_name in os.listdir(class_dir):
                if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                   self.image_paths.append(os.path.join(class_dir,image_name))
                   self.labels.append(0 if label == 'NORMAL' else 1)
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self,idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        
        return image, label


#data augmentation

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_dataset = PneumoniaDataset(root_dir='chest_xray/train',transform = train_transform)
test_dataset = PneumoniaDataset(root_dir='chest_xray/test',transform = test_transform)
val_dataset = PneumoniaDataset(root_dir='chest_xray/val',transform = test_transform)

class_counts = [train_dataset.labels.count(0), train_dataset.labels.count(1)]
class_weights = torch.tensor([1.0 / class_counts[0], 1.0 / class_counts[1]], device=device)

train_loader = DataLoader(train_dataset,batch_size = 32,shuffle = True)
test_loader = DataLoader(test_dataset,batch_size = 32,shuffle = False)
val_loader = DataLoader(val_dataset,batch_size = 32,shuffle = False)


model = models.resnet18(weights = models.ResNet18_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(model.fc.in_features,2)
model = model.to(device)


criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(),lr = 0.001)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.2, patience=2)



class EarlyStopping:
    def __init__(self, patience=5, verbose=False):
        self.patience = patience
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.verbose = verbose
        self.best_model = None

    def __call__(self, val_loss, model):
        if self.best_loss is None or val_loss < self.best_loss:
            self.best_loss = val_loss
            self.best_model = model.state_dict()
            self.counter = 0
            if self.verbose:
                print("Validation loss decreased. Saving model...")
        else:
            self.counter += 1
            if self.verbose:
                print(f"No improvement in validation loss. Patience: {self.counter}/{self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True


num_epochs = 25
early_stopping = EarlyStopping(patience=5, verbose=True)

#train

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    avg_train_loss = running_loss / len(train_loader)

    model.eval()
    val_loss = 0.0
    val_labels = []
    val_predictions = []

    
#validation

    
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs,labels)
            val_loss += loss.item()


            _, predictions = torch.max(outputs,1)
            val_labels.extend(labels.cpu().numpy())
            val_predictions.extend(predictions.cpu().numpy())
    
    avg_val_loss = val_loss / len(val_loader)
    val_acc = accuracy_score(val_labels, val_predictions)
    
    print(f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {avg_train_loss:.4f} - Val Loss: {avg_val_loss:.4f} - Val Acc: {val_acc:.4f}")


    scheduler.step(avg_val_loss)

    early_stopping(avg_val_loss, model)

    if early_stopping.early_stop:
        print("Early stopping triggered.")
        break


#getting the best model


model.load_state_dict(early_stopping.best_model)

model.eval()
test_labels = []
test_predictions = []

#testing 

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predictions = torch.max(outputs,1)

        test_labels.extend(labels.cpu().numpy())
        test_predictions.extend(predictions.cpu().numpy())

#measuring the accuracy

test_acc = accuracy_score(test_labels, test_predictions)
test_precision = precision_score(test_labels, test_predictions)
test_recall = recall_score(test_labels, test_predictions)
test_f1 = f1_score(test_labels, test_predictions)


print(f"\nTest Accuracy: {test_acc:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")
print(f"F1 Score: {test_f1:.4f}")
print("Confusion Matrix:\n", confusion_matrix(test_labels, test_predictions))
print("Classification Report:\n", classification_report(test_labels, test_predictions))


torch.save(model.state_dict(),'pneumonia_detection_model.pth')




Epoch [1/25] - Train Loss: 0.1833 - Val Loss: 1.4953 - Val Acc: 0.6875
Validation loss decreased. Saving model...
Epoch [2/25] - Train Loss: 0.1101 - Val Loss: 0.2636 - Val Acc: 0.8750
Validation loss decreased. Saving model...
Epoch [3/25] - Train Loss: 0.0934 - Val Loss: 1.9482 - Val Acc: 0.6250
No improvement in validation loss. Patience: 1/5
Epoch [4/25] - Train Loss: 0.0941 - Val Loss: 0.1112 - Val Acc: 1.0000
Validation loss decreased. Saving model...
Epoch [5/25] - Train Loss: 0.0819 - Val Loss: 0.3645 - Val Acc: 0.8750
No improvement in validation loss. Patience: 1/5
Epoch [6/25] - Train Loss: 0.0752 - Val Loss: 0.1334 - Val Acc: 0.8750
No improvement in validation loss. Patience: 2/5
Epoch [7/25] - Train Loss: 0.0538 - Val Loss: 0.2259 - Val Acc: 0.9375
No improvement in validation loss. Patience: 3/5
Epoch [8/25] - Train Loss: 0.0378 - Val Loss: 0.1187 - Val Acc: 0.9375
No improvement in validation loss. Patience: 4/5
Epoch [9/25] - Train Loss: 0.0415 - Val Loss: 0.2801 - Val

In [ ]:

model.eval()  

dummy_input = torch.randn(1, 3, 224, 224, device=device)  

onnx_path = "pneumonia_model.onnx"

torch.onnx.export(
    model,
    dummy_input,
    onnx_path,
    export_params=True,
    opset_version=11,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
)

print(f" Model exported to ONNX format at '{onnx_path}'")
